# Create RAG Prompt Template


This notebook should be run using with Runtime 22.2 & Python 3.10 or greater runtime environment, if you are viewing this in Watson Studio, and do not see Python 3.10.x or greater in the upper right corner of your screen, please update the runtime now. 

The notebook will create a RAG prompt template asset in a given project.


Note : User can search for `EDIT THIS` and fill the inputs needed.

## Prerequisite

* It requires service credentials for IBM Watson OpenScale/watsonx
* Requires the ID of project in which you want to create the prompt template asset.

### Contents

- [Setup](#settingup)
- [Create Prompt template](#prompt)


## Setup <a name="settingup"></a>

In [ ]:
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1
!pip install --upgrade evaluate --no-cache | tail -n 1
#!pip install --upgrade --extra-index-url https://test.pypi.org/simple/ ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1
!pip install --upgrade matplotlib | tail -n 1
!pip install --upgrade pydantic==1.10.11 --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade ibm_aigov_facts_client --no-cache | tail -n 1
!pip install --upgrade openai --no-cache | tail -n 1 
!pip install --upgrade azure.identity --no-cache | tail -n 1

Note: you may need to restart the kernel to use updated packages.

### Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [ ]:
use_cpd = False

if use_cpd:
    CPD_URL = "https://cpd-cpd-instance.apps.gpu01.pok-lb.techzone.ibm.com/"
    CPD_USERNAME = "EDIT THIS"
    CPD_APIKEY = "EDIT THIS"
    WOS_SERVICE_INSTANCE_ID = "None" # If None, default instance would be used
else:
    IAM_URL = "https://iam.cloud.ibm.com"
    DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
    SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
    CLOUD_API_KEY = "<EDIT THIS>" # YOUR_CLOUD_API_KEY
    

## Set the project ID

In order to set up a development type subscription, the PTA must be within the project. Please supply the project ID where the PTA needs to be created.

In [ ]:
PROJECT_ID = "<EDIT THIS>" # YOUR_PROJECT_ID

## Function to create the access token

This function generates an IAM access token using the provided credentials. The API calls for creating and scoring prompt template assets utilize the token generated by this function.

In [ ]:
import requests, json
def generate_access_token():
    headers={}
    if not use_cpd: 
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        headers["Accept"] = "application/json"
        data = {
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": CLOUD_API_KEY,
            "response_type": "cloud_iam"
        }
        response = requests.post(IAM_URL + "/identity/token", data=data, headers=headers)
        json_data = response.json()
        iam_access_token = json_data["access_token"]
    else:
        headers["Content-Type"] = "application/json"
        headers["Accept"] = "application/json"
        data = {
            "username": CPD_USERNAME,
            "api_key": CPD_APIKEY
        }
        data = json.dumps(data).encode("utf-8")
        url = CPD_URL + "/icp4d-api/v1/authorize"
        response = requests.post(url=url, data=data, headers=headers, verify=False)
        json_data = response.json()
        iam_access_token = json_data["token"]
        
    return iam_access_token

iam_access_token = generate_access_token()

In [ ]:
if not use_cpd:
    credentials={
        "apikey": CLOUD_API_KEY,
        "url": "https://us-south.ml.cloud.ibm.com"
    }

# Create Prompt template <a name="prompt"></a>

In [ ]:
from ibm_watson_machine_learning.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

prompt_mgr = PromptTemplateManager(
                credentials = credentials,
                project_id = PROJECT_ID
                )

prompt_template = PromptTemplate(name="RAG QA",
                                 model_id=ModelTypes.GRANITE_13B_CHAT_V2,
                                 task_ids=["retrieval_augmented_generation"],
                                 input_prefix="",
                                 output_prefix="",
                                 input_text="Answer the below question from the given context only and do not use the knowledge outside the context.\n\nContext: {context1} {context2} {context3} {context4}\nQuestion: {question}\nAnswer:",
                                 input_variables=["context1", "context2", "context3", "context4", "question"])

stored_prompt_template = prompt_mgr.store_prompt(prompt_template)
project_pta_id = stored_prompt_template.prompt_id
project_pta_id

Create a detached prompt template for a RAG task

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

service_instance_id = None # Update this to refer to a particular service instance
authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY, url = IAM_URL)
wos_client = APIClient(authenticator=authenticator, service_url = SERVICE_URL, service_instance_id = service_instance_id)
print(wos_client.version)

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_mart_id = "d2af45ac-3ec7-40ac-ab47-970c6c7dd4de"

In [ ]:
# Update the label_column, context_fields, question_field values based on the prompt and test data used
label_column = "answer"
context_fields = ["context1", "context2", "context3", "context4"]
question_field = "question"

operational_space_id = "development"
problem_type= "retrieval_augmented_generation"
input_data_type= "unstructured_text"


monitors = {
    "generative_ai_quality": {
        "parameters": {
            "min_sample_size": 5,
            "metrics_configuration":{
                "faithfulness": {
                    #"attributions_count": 3,
                    #"ngrams": 2,
                },
                "answer_relevance": {},
                "rouge_score": {},
                "exact_match": {},
                "bleu": {},
                "unsuccessful_requests": {
                    #"unsuccessful_phrases": []
                },
                "hap_input_score": {},
                "hap_score": {},
                "pii": {},
                "pii_input": {}
            }
        }
    }
}

response = wos_client.wos.execute_prompt_setup(prompt_template_asset_id = project_pta_id, 
                                               project_id = PROJECT_ID,
                                               context_fields = context_fields,
                                               question_field = question_field,
                                               label_column = label_column,
                                               operational_space_id = operational_space_id, 
                                               problem_type = problem_type,
                                               input_data_type = input_data_type, 
                                               supporting_monitors = monitors, 
                                               background_mode = False)

result = response.result
result._to_dict()

## Congratulations!

You have finished creating a prompt template! You can now navigate to the prompt template asset in your project and evaluate and track to a use case.

In [ ]:
response = wos_client.wos.get_prompt_setup(prompt_template_asset_id = project_pta_id,
                                                             project_id = PROJECT_ID)

result = response.result
result_json = result._to_dict()

if result_json["status"]["state"] == "FINISHED":
    print("Finished prompt setup : The response is {}".format(result_json))
else:
    print("prompt setup failed The response is {}".format(result_json))

In [ ]:
dev_subscription_id = result_json["subscription_id"]
dev_subscription_id

In [ ]:
# Download rag data
!rm rag_state_union.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/watsonx/rag_state_union.csv

In [ ]:
test_data_path = "rag_state_union.csv"
body = None # Please update your mapping file path here if needed

# Download data from project to local directory
# Run the below code snippet only if you are running the notebook via Watson Studio
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
wslib.download_file(test_data_path)
if body:
    wslib.download_file(body)

In [ ]:
monitor_definition_id = "mrm"
target_target_id = dev_subscription_id
result = wos_client.monitor_instances.list(data_mart_id=data_mart_id,
                                           monitor_definition_id=monitor_definition_id,
                                           target_target_id=target_target_id,
                                           project_id=PROJECT_ID).result
result_json = result._to_dict()
mrm_monitor_id = result_json["monitor_instances"][0]["metadata"]["id"]
mrm_monitor_id

In [ ]:
test_data_set_name = "data"
content_type = "multipart/form-data"

response  = wos_client.monitor_instances.mrm.evaluate_risk(monitor_instance_id=mrm_monitor_id, 
                                                    test_data_set_name = test_data_set_name, 
                                                    test_data_path = test_data_path,
                                                    content_type = content_type,
                                                    body = body,
                                                    project_id = PROJECT_ID,
                                                    background_mode = False)

In [ ]:
response  = wos_client.monitor_instances.mrm.get_risk_evaluation(mrm_monitor_id, project_id = PROJECT_ID)
response.result.to_dict()

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=mrm_monitor_id, project_id=PROJECT_ID)

In [ ]:
monitor_definition_id = "generative_ai_quality"
result = wos_client.monitor_instances.list(data_mart_id = data_mart_id,
                                           monitor_definition_id = monitor_definition_id,
                                           target_target_id = target_target_id,
                                           project_id = PROJECT_ID).result
result_json = result._to_dict()
genaiquality_monitor_id = result_json["monitor_instances"][0]["metadata"]["id"]
genaiquality_monitor_id

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=genaiquality_monitor_id, project_id=PROJECT_ID)

In [ ]:
result = wos_client.data_sets.list(target_target_id = dev_subscription_id,
                                target_target_type = "subscription",
                                type = "gen_ai_quality_metrics").result

genaiq_dataset_id = result.data_sets[0].metadata.id
genaiq_dataset_id

In [ ]:
wos_client.data_sets.show_records(data_set_id = genaiq_dataset_id)

In [ ]:
factsheets_url = factsheets_url = "{}/wx/prompt-details/{}/factsheet?context=wx&project_id={}".format("https://dataplatform.cloud.ibm.com",project_pta_id, PROJECT_ID)
print("User can navigate to the published facts in project {}".format(factsheets_url))